In [ ]:
using Pkg; Pkg.activate(".")
using CairoMakie, LinearAlgebra, LaTeXStrings, FFTA

In [ ]:
"interpolation nodes"
xgrid(N) = [ j * π / N  for j = 0:2N-1 ]

"fourier coefficient indices"
kgrid(N) = [ 0:N; -N+1:-1 ]

"""
construct the coefficients of the trigonometric interpolant
"""
function triginterp_fft(f::Function, N)
    X = xgrid(N)
    # nodal values at interpolation nodes
    F = f.(X) 
    return fft(F) / (2*N)
end 


"""
to evaluate a trigonometric polynomial just sum coefficients * basis
we the take the real part because we assume the function we are 
approximating is real.
"""
evaltrig(x, F̂) = sum( real(F̂k * exp(im * x * k))
                      for (F̂k, k) in zip(F̂, kgrid(length(F̂) ÷ 2)) )
;

We return to our periodic witch of Agnesi example.

In [ ]:
f, α = let c = 3
    (x -> 1 / (1 + c^2 * sin(x/2)^2)), 2*asinh(1/c)
end

fig1 = Figure(size = (400, 250)); ax1 = Axis(fig1[1,1])
lines!(ax1, -π..π, f); fig1

In [ ]:
NN = [4, 8, 12, 16, 20, 24, 28]
# NN = 8:8:128
xe = range(-π, π, length=1000)
f_ex = f.(xe)
err_max = zeros(length(NN))
for (i, N) in enumerate(NN)
    F̂ = triginterp_fft(f, N);
    fN = evaltrig.(xe, Ref(F̂))
    err_max[i] = norm(f_ex - fN, Inf)
end


In [ ]:
fig2 = Figure(size = (400, 300))
ax2 = Axis(fig2[1, 1], xlabel = L"N", ylabel = "", yscale = log10)
scatterlines!(ax2, NN, err_max, label=L"\Vert f - I_N f \Vert_\infty")
lines!(ax2, NN[4:end], 2*exp.(-α * NN[4:end]), linestyle=:dash, color=:black, label=L"e^{-\alpha N}")
axislegend(ax2, position=:rt)
# fig2
fig2

Now that we have a fast algorithm, we can make the problem a bit harder.

In [ ]:
f, α = let c = 20
    (x -> 1 / (1 + c^2 * sin(x/2)^2)), 2*asinh(1/c)
end

NN = 16:32:512
xe = range(-π, π, length=4000)
f_ex = f.(xe)
err_max = zeros(length(NN))
for (i, N) in enumerate(NN)
    F̂ = triginterp_fft(f, N);
    fN = evaltrig.(xe, Ref(F̂))
    err_max[i] = norm(f_ex - fN, Inf)
end


In [ ]:
fig3 = Figure(size = (400, 300))
ax3 = Axis(fig3[1, 1], xlabel = L"N", ylabel = "", yscale = log10)
scatterlines!(ax3, NN, err_max, label=L"\Vert f - I_N f \Vert_\infty")
lines!(ax3, NN[4:10], 10*exp.(-α * NN[4:10]), linestyle=:dash, color=:black, label=L"\sim e^{-\alpha N}")
axislegend(ax3, position=:rt)
fig3

To estimate the errors we are evaluating the trigonometric interpolant on an increasing number of grid points. That still has quadratic scaling! But this limitation can also be removed.

### Evaluating Trigonometric Polynomials on a Grid

The key observation is that the IFFT can be used to obtain the nodal values. So to get nodal values on a refined grid we can simply "pad" the vector $\hat{F}$ with zeros. This is what `evaltrig_grid` does.

In [ ]:
function evaltrig_grid(F̂, M::Integer; convert = real)
    N = length(F̂) ÷ 2
    @assert 2 * N == length(F̂)
    @assert M >= N 
    F̂_M = zeros(ComplexF64, 2*M)
    F̂_M[1:N] .= F̂[1:N]
    F̂_M[end-N+1:end] .= F̂[end-N+1:end]
    x = xgrid(M) 
    Fx = real.(ifft(F̂_M) * (2*M))
    return Fx, x
end

In [ ]:
N = 3
F̂ = randn(2*N)

# naive plotting just the 6 nodal values?
xp1 = [xgrid(N); [2*π]]
F1 = real.(ifft(F̂)) * (2*N); push!(F1, F1[1])

# We can of course just use eval_trig on a finer grid
xp2 = range(0, 2*π, length=200)
F2 = evaltrig.(xp2, Ref(F̂))

# Nlog(N) scaling cost 
F3, xp3 = evaltrig_grid(F̂, 200)
push!(F3, F3[1]); xp3 = [xp3; [2*π]]

fig4 = Figure(size = (400, 300)); ax4 = Axis(fig4[1,1])
lines!(ax4, xp1, F1, label = "pw")
lines!(ax4, xp2, F2, label = "evaltrig")
lines!(ax4, xp3, F3, label = "evaltrig_grid", linestyle = :dash)
fig4

Equipped with this technique we can now easily scale the error analysis to arbitarily large problems. 

In [ ]:
f, α = let c = 100
    (x -> 1 / (1 + c^2 * sin(x/2)^2)), 2*asinh(1/c)
end

fig5 = Figure(size = (400, 250)); ax5 = Axis(fig5[1,1])
lines!(ax5, -π..π, f); fig5

In [ ]:

# NN = 16:32:512
NN = 128:128:(16*128)   # 16 * 128 = 2048
err_max = zeros(length(NN))
Nerr = 2^15  # ≈ 32000
for (i, N) in enumerate(NN)
    F̂ = triginterp_fft(f, N);
    Fe, xe = evaltrig_grid(F̂, Nerr)
    err_max[i] = norm(real.(Fe) - f.(xe), Inf)
end


In [ ]:
fig6 = Figure(size = (400, 300))
ax6 = Axis(fig6[1, 1], xlabel = L"N", ylabel = "", yscale = log10)
scatterlines!(ax6, NN, err_max, label=L"\Vert f - I_N f \Vert_\infty")
lines!(ax6, NN[4:10], 10*exp.(-α * NN[4:10]), linestyle=:dash, color=:black, label=L"\sim e^{-\alpha N}")
axislegend(ax6, position=:rt)
fig6

---

Experiment?

In [ ]:
function interpolation_error(f, N::Integer, Nerr = N * 8)
    F̂ = triginterp_fft(f, N);
    Fe, xe = evaltrig_grid(F̂, Nerr)
    return norm(real.(Fe) - f.(xe), Inf)
end

In [ ]:
# Example: Fermi-Dirac Function

# define a new function
f = x -> 1 / (1 + exp(20*sin(x)))
α = π/20

# plot the target function
p1 = plot(f, -π/2, π/2, lw=3, label = "f", xlabel = "x")

# compute errors
NN = 16:16:(16*16); Nerr = 4092
max_err = interpolation_error.(Ref(f), NN, Nerr)

# plot errors 
p2 = plot(NN, max_err; lw=3, m=:o, ms=5, label = L"\|f - I_N f\|_\infty",
    yscale = :log10, xscale = :identity, xlabel = L"N", 
    size = (400, 300), ylims = (1e-16, 1.0))
plot!(p2, NN[4:end], 8*exp.(-α * NN[4:end]); lw=2, ls=:dash, c = :black, 
        label = L"e^{-\alpha N}")

# plot both 
plot(p1, p2, layout = (1,2), size = (600, 300))

In [ ]:
# Example: A Singular function C^{2,1} - regularity
#     not analytic so we only get algebraic decay.

# define a new function
f = x -> abs(sin(x))^3
α = 3.0

# plot the target function
p1 = plot(f, -π/2, π/2, lw=3, label = "f", xlabel = "x")

# compute errors
NN = (2).^(3:2:16); Nerr = 8 * NN[end]
max_err = interpolation_error.(Ref(f), NN, Nerr)

# plot errors 
p2 = plot(NN, max_err; lw=3, m=:o, ms=5, label = L"\|f - I_N f\|_\infty",
    yscale = :log10, xscale = :log10, xlabel = L"N", 
    size = (400, 300), ylims = (1e-16, 1.0))
plot!(p2, NN[4:end], 10*(NN[4:end]).^(-α); lw=2, ls=:dash, c = :black, 
        label = L"N^{-\alpha}")

# plot both 
plot(p1, p2, layout = (1,2), size = (600, 300))


---

In [ ]:
# define a new function
f = x -> abs(sin(x))^(2+1/3)
α = 2+1/3

# plot the target function
p1 = plot(f, -π/2, π/2, lw=3, label = "f", xlabel = "x")

# compute errors
NN = (2).^(3:2:16); Nerr = 8 * NN[end]
max_err = interpolation_error.(Ref(f), NN, Nerr)

# plot errors 
p2 = plot(NN, max_err; lw=3, m=:o, ms=5, label = L"\|f - I_N f\|_\infty",
    yscale = :log10, xscale = :log10, xlabel = L"N", 
    size = (400, 300), ylims = (1e-16, 1.0))
plot!(p2, NN[4:end], 1*(NN[4:end]).^(-α); lw=2, ls=:dash, c = :black, 
        label = L"N^{-\alpha}")

# plot both 
plot(p1, p2, layout = (1,2), size = (600, 300))


In [ ]:
# define a new function
function finf(x)
    if 0 < x < π 
        return exp(-1/sin(x)) 
    else
        return zero(typeof(x))
    end
end

# plot the target function
p1 = plot(finf, -π, π, lw=3, label = "f", xlabel = "x")

# compute errors
NN1 = 16:16:256; Nerr = 8 * NN1[end]
max_err1 = interpolation_error.(Ref(finf), NN1, Nerr)
p2 = plot(NN1, max_err1; lw=3, m=:o, ms=5, label = L"\|f - I_N f\|_\infty",
    yscale = :log10, xscale = :identity, xlabel = L"N", 
    size = (400, 300), )

NN2 = (2).^(3:12); Nerr = 8 * NN2[end]
max_err2 = interpolation_error.(Ref(finf), NN2, Nerr)
p3 = plot(NN2, max_err2; lw=3, m=:o, ms=5, label = L"\|f - I_N f\|_\infty",
    yscale = :log10, xscale = :log10, xlabel = L"N", 
    size = (400, 300), )

# plot both 
plot(p2, p3, layout = (1,2), size = (600, 300))
